In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
df = pd.read_csv("D:\Redi_DataCircle2\survey_results_public.csv")

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89184 entries, 0 to 89183
Data columns (total 84 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ResponseId                           89184 non-null  int64  
 1   Q120                                 89184 non-null  object 
 2   MainBranch                           89184 non-null  object 
 3   Age                                  89184 non-null  object 
 4   Employment                           87898 non-null  object 
 5   RemoteWork                           73810 non-null  object 
 6   CodingActivities                     73764 non-null  object 
 7   EdLevel                              87973 non-null  object 
 8   LearnCode                            87663 non-null  object 
 9   LearnCodeOnline                      70084 non-null  object 
 10  LearnCodeCoursesCert                 37076 non-null  object 
 11  YearsCode                   

In [20]:
can write a function that look for some words and create a boolean column
LearnCode
LearnCodeOnline
df["LearnCodeCoursesCert"]
df["LanguageHaveWorkedWith"]
DatabaseHaveWorkedWith
PlatformHaveWorkedWith
WebframeHaveWorkedWith
MiscTechHaveWorkedWith
ToolsTechHaveWorkedWith
NEWCollabToolsHaveWorkedWith
 SurveyLength                         86485 non-null  object 
 82  SurveyEase                           86554 non-null  object 
 83  ConvertedCompYearly

NameError: name 'LearnCode' is not defined

In [49]:
df["ConvertedCompYearly"].value_counts()#.head(40)

ConvertedCompYearly
64254.0     784
53545.0     615
150000.0    585
200000.0    558
74963.0     557
           ... 
31494.0       1
859.0         1
77904.0       1
74917.0       1
16917.0       1
Name: count, Length: 8784, dtype: int64

In [50]:
df = df.dropna(subset=['ConvertedCompYearly'])

In [ ]:
check CompTotal  if  is the same as if df["ConvertedCompYearly"] and df["Currency"]. look for outliers, mistakes, not correct convertations and other animalies

In [71]:
df["ProfessionalTech"].value_counts()

ProfessionalTech
None of these                                                                                                                                                                                           3346
DevOps function;Microservices;Automated testing;Observability tools;Developer portal or other central places to find tools/services;Continuous integration (CI) and (more often) continuous delivery    2114
DevOps function;Microservices;Automated testing;Observability tools;Continuous integration (CI) and (more often) continuous delivery                                                                    1881
DevOps function;Microservices;Automated testing;Continuous integration (CI) and (more often) continuous delivery                                                                                        1298
DevOps function;Automated testing;Continuous integration (CI) and (more often) continuous delivery                                                                 

In [ ]:
#Check if important for salary
df["RemoteWork"]
df["Age"] - categorize
df["Employment"] - simplify
df["EdLevel"]
df["YearsCode"] or "YearsCodePro" before change rthe columns less then or more then  (should be normalized or categorized for ML)
"YearsCodePro" make more sence cause its exp without studying. check if its always less or not. also  check with df["WorkExp"]  and choose one of them
df["DevType"] question is if there is enought for some special roles
df["OrgSize"] categorise
df["PurchaseInfluence"] categorise
df["Country"] - group firstly to bigger groups (EU, EastEurope, others)
df["ProfessionalTech"] group firstly 
df["Industry"]  

In [13]:
df["Age"].value_counts()

Age
25-34 years old       33247
35-44 years old       20532
18-24 years old       17931
45-54 years old        8334
Under 18 years old     4128
55-64 years old        3392
65 years or older      1171
Prefer not to say       449
Name: count, dtype: int64

In [53]:
df["Knowledge_1"].value_counts()

Knowledge_1
Agree                         14873
Strongly agree                12121
Neither agree nor disagree     2477
Disagree                       1949
Strongly disagree               752
Name: count, dtype: int64

In [4]:
def drop_from_df(df, labels, axis=1):
    return df.drop(labels, axis=axis, errors='ignore')

In [52]:
df = drop_from_df(df, ['Q120', 'TechList','BuyNewTool','SurveyLength','SurveyEase'])

In [54]:
replacement_map = {
    "I am a developer by profession": "dev",
    "I am not primarily a developer, but I write code sometimes as part of my work/studies": "code, but not dev",
    "I am learning to code": "learning",
    "I code primarily as a hobby": "hobby",
    "I used to be a developer by profession, but no longer am": "was developer",
    "None of these": "other"
}
df['MainBranch'] = df['MainBranch'].replace(replacement_map)

In [25]:
df["CompTotal"].value_counts()#.head(40) - do a visualisation of distribution

CompTotal
100000.0     1355
60000.0      1290
120000.0     1240
150000.0     1150
80000.0      1066
             ... 
165.0           1
182700.0        1
2810000.0       1
49280.0         1
182333.0        1
Name: count, Length: 3797, dtype: int64

In [44]:
We can give a points depends from the answer and summarise all answers. 
Afterwards categories and check if there is a corellation with salary

For example
Agree"5"
Strongly agree "4"
Neither agree nor disagree "3"
Disagree "2"
Strongly disagree "1"
NaN 0
Max = 40 points min. 0
Knowledge_1                          42535 non-null  object 
 67  Knowledge_2                          41670 non-null  object 
 68  Knowledge_3                          41798 non-null  object 
 69  Knowledge_4                          41684 non-null  object 
 70  Knowledge_5                          41527 non-null  object 
 71  Knowledge_6                          41520 non-null  object 
 72  Knowledge_7                          41467 non-null  object 
 73  Knowledge_8  

SyntaxError: invalid syntax (217119686.py, line 1)

In [ ]:
Frequency 1. Needing help from people outside of your immediate team?  
2. Interacting with people outside of your immediate team?  

3. Encountering knowledge silos (where one individual or team has information that's not
shared or distributed with other individuals or teams) at work?


In [46]:
display (df["Frequency_1"].value_counts()
         , df["Frequency_2"].value_counts()
         , df["Frequency_3"].value_counts()
         )

Frequency_1
1-2 times a week     25528
Never                10707
3-5 times a week      4100
6-10 times a week      847
10+ times a week       734
Name: count, dtype: int64

Frequency_2
1-2 times a week     18930
3-5 times a week      9809
10+ times a week      4794
6-10 times a week     4383
Never                 4009
Name: count, dtype: int64

Frequency_3
1-2 times a week     21470
Never                12107
3-5 times a week      5125
6-10 times a week     1280
10+ times a week      1072
Name: count, dtype: int64

In [6]:
df["OrgSize"].value_counts()

OrgSize
20 to 99 employees                                    13380
100 to 499 employees                                  12218
10,000 or more employees                               7929
1,000 to 4,999 employees                               7235
2 to 9 employees                                       6439
10 to 19 employees                                     5254
500 to 999 employees                                   4472
Just me - I am a freelancer, sole proprietor, etc.     4196
5,000 to 9,999 employees                               2677
I don’t know                                           1243
Name: count, dtype: int64

In [7]:
df["Employment"].value_counts()

Employment
Employed, full-time                                                                                                                                                                                             53748
Student, full-time                                                                                                                                                                                               7430
Independent contractor, freelancer, or self-employed                                                                                                                                                             7076
Employed, full-time;Independent contractor, freelancer, or self-employed                                                                                                                                         4354
Not employed, but looking for work                                                                                                   

In [55]:
def calculate_knowledge_score(df):
    answer_points = {
        "Strongly agree": 5,
        "Agree": 4,
        "Neither agree nor disagree": 3,
        "Disagree": 2,
        "Strongly disagree": 1,
        pd.NA: 0, 
    }

    knowledge_cols = [col for col in df if col.startswith("Knowledge_")]

    # Map answers to points and sum to create 'Knowledge_Score'
    for col in knowledge_cols:
        df[col] = df[col].map(answer_points)

    df['Knowledge_Score'] = df[knowledge_cols].sum(axis=1)
    
    return df

In [60]:
df['Knowledge_Score'].value_counts()

Knowledge_Score
0.0     15809
28.0     3801
29.0     3649
27.0     3601
30.0     3318
26.0     3186
31.0     2466
25.0     2329
24.0     1961
32.0     1815
23.0     1130
33.0     1098
22.0      775
34.0      591
21.0      489
35.0      339
20.0      303
36.0      194
19.0      173
4.0       131
18.0       99
37.0       84
40.0       63
17.0       59
16.0       57
5.0        55
38.0       47
8.0        47
3.0        39
2.0        36
12.0       34
6.0        28
7.0        28
9.0        27
11.0       25
39.0       25
10.0       23
15.0       23
14.0       22
1.0        21
13.0       19
Name: count, dtype: int64

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48019 entries, 1 to 89171
Data columns (total 80 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ResponseId                           48019 non-null  int64  
 1   MainBranch                           48019 non-null  object 
 2   Age                                  48019 non-null  object 
 3   Employment                           48007 non-null  object 
 4   RemoteWork                           47940 non-null  object 
 5   CodingActivities                     47918 non-null  object 
 6   EdLevel                              48019 non-null  object 
 7   LearnCode                            47935 non-null  object 
 8   LearnCodeOnline                      38414 non-null  object 
 9   LearnCodeCoursesCert                 20744 non-null  object 
 10  YearsCode                            47950 non-null  object 
 11  YearsCodePro                     

In [58]:
calculate_knowledge_score(df)

,ResponseId,MainBranch,Age,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,LearnCodeCoursesCert,...,Knowledge_8,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,ProfessionalTech,Industry,ConvertedCompYearly,Knowledge_Score
1,2,dev,25-34 years old,"Employed, full-time",Remote,Hobby;Contribute to open-source projects;Boots...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;Friend or fam...,Formal documentation provided by the owner of ...,Other,...,5.0,1-2 times a week,10+ times a week,Never,15-30 minutes a day,15-30 minutes a day,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",285000.0,35.0
2,3,dev,45-54 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;Professional development or self-paced l...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Formal documentation provided by the owner of ...,NaN,...,4.0,6-10 times a week,6-10 times a week,3-5 times a week,30-60 minutes a day,30-60 minutes a day,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",250000.0,32.0
3,4,dev,25-34 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Colleague;Friend or family member;Other online...,Formal documentation provided by the owner of ...,NaN,...,4.0,1-2 times a week,10+ times a week,1-2 times a week,15-30 minutes a day,30-60 minutes a day,Automated testing;Continuous integration (CI) ...,NaN,156000.0,31.0
4,5,dev,25-34 years old,"Employed, full-time;Independent contractor, fr...",Remote,Hobby;Contribute to open-source projects;Profe...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Online Courses or Certi...,Formal documentation provided by the owner of ...,Other;Codecademy;edX,...,4.0,1-2 times a week,1-2 times a week,3-5 times a week,60-120 minutes a day,30-60 minutes a day,Microservices;Automated testing;Observability ...,Other,23456.0,33.0
5,6,dev,35-44 years old,"Employed, full-time",Remote,Hobby;Professional development or self-paced l...,Some college/university study without earning ...,Books / Physical media;Colleague;Online Course...,Formal documentation provided by the owner of ...,Other,...,2.0,1-2 times a week,1-2 times a week,3-5 times a week,30-60 minutes a day,15-30 minutes a day,DevOps function;Microservices;Observability to...,Other,96828.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89162,89163,dev,35-44 years old,"Employed, full-time",Remote,I don’t code outside of work,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",On the job training;Other online resources (e....,Formal documentation provided by the owner of ...,NaN,...,3.0,6-10 times a week,6-10 times a week,3-5 times a week,60-120 minutes a day,60-120 minutes a day,Automated testing;Developer portal or other ce...,"Information Services, IT, Software Development...",50719.0,28.0
89166,89167,dev,18-24 years old,"Employed, full-time;Independent contractor, fr...",In-person,Contribute to open-source projects;Professiona...,Some college/university study without earning ...,Books / Physical media;Hackathons (virtual or ...,Formal documentation provided by the owner of ...,Codecademy;Udemy,...,3.0,1-2 times a week,1-2 times a week,1-2 times a week,15-30 minutes a day,15-30 minutes a day,DevOps function;Microservices;Continuous integ...,Financial Services,16917.0,28.0
89167,89168,dev,18-24 years old,"Employed, full-time",Remote,Hobby;Bootstrapping a business;Professional de...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Online Courses or Certification;Other online r...,Formal documentation provided by the owner of ...,edX;Udemy;Coursera,...,4.0,1-2 times a week,1-2 times a week,NaN,Over 120 minutes a day,30-60 minutes a day,Automated testing,Retail and Consumer Services,15752.0,31.0


In [61]:
df = drop_from_df(df, ['Knowledge_1', 'Knowledge_2','Knowledge_3','Knowledge_4','Knowledge_5','Knowledge_6','Knowledge_7','Knowledge_8'])

In [ ]:
TimeSearching                        42778 non-null  object 
TimeAnswering

In [32]:
df['TimeSearching'].value_counts()

TimeSearching
30-60 minutes a day           12382
15-30 minutes a day            8902
60-120 minutes a day           5759
Less than 15 minutes a day     2877
Over 120 minutes a day         2268
Name: count, dtype: int64

In [62]:
df['Country'].value_counts()

Country
United States of America                                11881
Germany                                                  3970
United Kingdom of Great Britain and Northern Ireland     3544
Canada                                                   2105
India                                                    1868
                                                        ...  
Gabon                                                       1
Guinea-Bissau                                               1
Mauritania                                                  1
Botswana                                                    1
Antigua and Barbuda                                         1
Name: count, Length: 171, dtype: int64

In [63]:
grouped_data = df.groupby('Country')['ConvertedCompYearly'].agg(['mean', 'median', 'count'])

grouped_data.sort_values(by='count', ascending=False).head(50)

,mean,median,count
Country,,,
United States of America,170783.961199,150000.0,11881
Germany,82457.479093,74963.0,3970
United Kingdom of Great Britain and Northern Ireland,95948.670429,81931.0,3544
Canada,142136.677435,89222.0,2105
India,27346.822805,19023.0,1868
France,65101.007773,56757.0,1801
Netherlands,85128.483847,74963.0,1362
Poland,64335.517378,57518.5,1266
Brazil,52883.154952,30431.0,1252


In [64]:
grouped_data.sort_values(by='mean', ascending=False).head(50)

,mean,median,count
Country,,,
Mali,9.156200e+06,25807.5,4
Afghanistan,8.436361e+05,915.0,11
Ethiopia,6.943592e+05,7004.5,26
Thailand,6.280319e+05,34508.0,104
Nomadic,4.331968e+05,102000.0,18
South Africa,2.595359e+05,41497.0,351
Australia,1.726176e+05,95851.0,1211
United States of America,1.707840e+05,150000.0,11881
Canada,1.421367e+05,89222.0,2105


In [65]:
df_GER = df[df['Country'] == 'Germany']
df_GER = df_GER[['Age','Employment','RemoteWork','EdLevel','YearsCodePro','DevType','OrgSize','ConvertedCompYearly','Knowledge_Score']]
df_GER = df_GER.rename(columns={'ConvertedCompYearly': 'Salary'})


In [69]:
df_GER = df_GER.dropna(subset=['RemoteWork', 'YearsCodePro', 'DevType', 'OrgSize'])

In [70]:
df_GER.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3942 entries, 15 to 89146
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Age              3942 non-null   object 
 1   Employment       3942 non-null   object 
 2   RemoteWork       3942 non-null   object 
 3   EdLevel          3942 non-null   object 
 4   YearsCodePro     3942 non-null   object 
 5   DevType          3942 non-null   object 
 6   OrgSize          3942 non-null   object 
 7   Salary           3942 non-null   float64
 8   Knowledge_Score  3942 non-null   float64
dtypes: float64(2), object(7)
memory usage: 308.0+ KB
